In [1]:
# Import selenium and its dependencies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
# import pandas 
import pandas as pd
import time
base_url = "https://members.ocpinfo.com/tcpr/public/pr/en/#/forms/new/?table=0x800000000000003D&form=0x800000000000002C&command=0x80000000000007C5&donotreload=ShowSRL"

scrape_df = pd.DataFrame()
scrape_df["Number of Corporations"] = 0


In [2]:
# launch the browser driver
driver = webdriver.Firefox()


In [4]:
# Loop through each page
for i in range(1, 2822):
    
    print(f"Page: {i}")
    
    # go the page url and wait 2 seconds for the page to load
    driver.get(f"{base_url}{i}")
    time.sleep(2)
    
    # check if the element is present
    try:
        element_present = EC.presence_of_element_located((By.ID, 'SearchReasultList'))
        WebDriverWait(driver, 5).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        
    # get the search results div
    search_divs = driver.find_element(By.ID, "SearchReasultList").find_elements(By.CLASS_NAME, "f15") 

    # loop through each entry in the div table
    for div in search_divs:
        
        # get the name and href of each pharmacist
        name = div.find_element(By.TAG_NAME, "a").text
        href =  div.find_element(By.TAG_NAME, "a").get_attribute("href")
        data = {
            "name": name,
            "href": href,
        }

        # append the data to the dataframe
        scrape_df = pd.concat([scrape_df, pd.DataFrame([data])], ignore_index=True)
    


Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7


KeyboardInterrupt: 

In [5]:
# for each entry in the dataframe go to their profile url
for index, row in scrape_df.iterrows():
    # go to there profile url
    driver.get(row["href"])
    
    # not all entries have a number of corporations so try and get it
    # other wise set it to 0
    try:
        scrape_df.iloc[index]["Number of Corporations"] = len(driver.find_element(By.ID, "OtherCorpoContent").find_elements(By.TAG_NAME, "div"))
    except NoSuchElementException:
        scrape_df.iloc[index]["Number of Corporations"] = 0
        pass

WebDriverException: Message: Failed to write request to stream
